In [1]:
; lazy infinite sequence with recursive definition
(def fib-seq-lazy 
  ((fn rfib [a b] 
     (lazy-seq (cons a (rfib b (+ a b)))))
   0 1))
(take 20 fib-seq-lazy)

clojure.lang.LazySeq@7c3704da

In [2]:
; 'unless' cannot be defined with a function because
; it does not always evaluate both its arguments.
(defmacro unless [pred a b]
  `(if (not ~pred) ~a ~b))
(unless false (println "Will print") (println "Will not print"))

Will print


null

In [3]:
(defn run [nvecs nitems nthreads niters]
  (let [vec-refs (vec (map (comp ref vec)
                           (partition nitems (range (* nvecs nitems)))))
        swap #(let [v1 (rand-int nvecs)
                    v2 (rand-int nvecs)
                    i1 (rand-int nitems)
                    i2 (rand-int nitems)]
                (dosync
                 (let [temp (nth @(vec-refs v1) i1)]
                   (alter (vec-refs v1) assoc i1 (nth @(vec-refs v2) i2))
                   (alter (vec-refs v2) assoc i2 temp))))
        report #(do
                 (prn (map deref vec-refs))
                 (println "Distinct:"
                          (count (distinct (apply concat (map deref vec-refs))))))]
    (report)
    (dorun (apply pcalls (repeat nthreads #(dotimes [_ niters] (swap)))))
    (report)))
; When run, we see no values get lost or duplicated in the shuffle.
; There are 36 distinct numbers before and after. 
(run 6 6 6 100000)

([0 1 2 3 4 5] [6 7 8 9 10 11] [12 13 14 15 16 17] [18 19 20 21 22 23] [24 25 26 27 28 29] [30 31 32 33 34 35])
Distinct: 36
([17 3 32 19 21 8] [29 22 4 0 30 24] [31 12 16 11 13 14] [26 20 10 6 1 15] [7 18 27 35 33 5] [34 25 9 23 2 28])
Distinct: 36


null